In [1]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_score 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import clone
from sklearn import linear_model
#silence future warning message
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [36]:
item = pd.read_csv('items.csv')

In [2]:
df = pd.read_csv('sales_train.csv')

In [64]:
df.head()

,key_0,date_block_num,shop_id,item_id_x,item_price,item_cnt_day,item_category_id
0,22154,0,59,22154,999.0,1.0,37
1,22154,0,24,22154,999.0,1.0,37
2,22154,0,27,22154,999.0,1.0,37
3,22154,0,25,22154,999.0,1.0,37
4,22154,0,25,22154,999.0,1.0,37


In [66]:
#df.drop(['item_name','item_id_y'], axis =1, inplace = True)
df = df.rename(columns={'item_id_x': 'item_id'})

In [56]:
#df = df.drop(['date'], axis = 1)

In [67]:
df.head()

,key_0,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,22154,0,59,22154,999.0,1.0,37
1,22154,0,24,22154,999.0,1.0,37
2,22154,0,27,22154,999.0,1.0,37
3,22154,0,25,22154,999.0,1.0,37
4,22154,0,25,22154,999.0,1.0,37


In [68]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]

rows_with_NaN.head(50)

,key_0,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id


In [69]:
def overview(dataframe):
    
    data_resumen = {'Valores Unicos':dataframe.nunique(),'No-Nulos': dataframe.notnull().sum(), 'Nulos': dataframe.isnull().sum(), 'Formato': dataframe.dtypes, 'Min': dataframe.min(), 'Max': dataframe.max()}
    resumen = pd.DataFrame(data=data_resumen)
    return resumen

In [70]:
overview(df)

,Valores Unicos,No-Nulos,Nulos,Formato,Min,Max
key_0,21803,2928489,0,int64,0.00,22169.0
date_block_num,34,2928489,0,int64,0.00,33.0
shop_id,60,2928489,0,int64,0.00,59.0
item_id,21803,2928489,0,int64,0.00,22169.0
item_price,19975,2928489,0,float64,0.07,59200.0
item_cnt_day,187,2928489,0,float64,1.00,669.0
item_category_id,84,2928489,0,int64,0.00,83.0


In [71]:
#Get rid of outliers
df = df[(df['item_price']<100000) & (df['item_price']>=0)]
df = df[(df['item_cnt_day']<1000) & (df['item_cnt_day']>=0)]

# plot after outliers removal
#plot_features = ['item_price','item_cnt_day']
#for f in plot_features:
#   df.boxplot(f,f)


In [85]:
df2 = df.groupby(by = ['date_block_num','shop_id','item_category_id','item_id','item_price'])['item_cnt_day'].sum()

In [86]:
df2 = df2.reset_index(drop=False)
df2['total_sales'] = df2['item_price'] * df2['item_cnt_day']
df2.head()

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,total_sales
0,0,0,2,5572,1322.0,10.0,13220.0
1,0,0,2,5573,560.0,1.0,560.0
2,0,0,2,5575,806.0,4.0,3224.0
3,0,0,2,5576,2231.0,5.0,11155.0
4,0,0,2,5609,2381.0,1.0,2381.0


In [87]:
df2['item_id_avg_item_price'] = df2.groupby(['date_block_num','shop_id','item_id'])['total_sales'].transform('mean')


In [88]:
df2.head(30)

,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,total_sales,item_id_avg_item_price
0,0,0,2,5572,1322.0,10.0,13220.0,13220.0
1,0,0,2,5573,560.0,1.0,560.0,560.0
2,0,0,2,5575,806.0,4.0,3224.0,3224.0
3,0,0,2,5576,2231.0,5.0,11155.0,11155.0
4,0,0,2,5609,2381.0,1.0,2381.0,2381.0
5,0,0,2,5612,3623.0,1.0,3623.0,3623.0
6,0,0,2,5623,294.0,1.0,294.0,294.0
7,0,0,2,5627,2060.0,2.0,4120.0,4120.0
8,0,0,2,5629,1925.0,9.0,17325.0,17325.0
9,0,0,2,5630,2060.0,1.0,2060.0,2060.0


In [89]:
df3 = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_id_avg_item_price'].mean()

In [90]:
df3 = df3.reset_index(drop=False)
df3.head(6)

,date_block_num,shop_id,item_id,item_id_avg_item_price
0,0,0,32,1326.0
1,0,0,33,1041.0
2,0,0,35,247.0
3,0,0,43,221.0
4,0,0,51,128.5
5,0,0,61,195.0


In [91]:
df3['item_id_sum_item_cnt_day'] = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('sum')
df3['item_id_avg_item_cnt_day'] = df2.groupby(by = ['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('mean')

In [92]:
df3.head()

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day
0,0,0,32,1326.0,10.0,10.0
1,0,0,33,1041.0,1.0,1.0
2,0,0,35,247.0,4.0,4.0
3,0,0,43,221.0,5.0,5.0
4,0,0,51,128.5,1.0,1.0


In [93]:
df3['shop_id_avg_item_price'] = df2.groupby(['date_block_num','shop_id'])['total_sales'].transform('mean')

df3['shop_id_avg_item_price'] = round(df3['shop_id_avg_item_price'],2)

In [94]:
df3.head(10)

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price
0,0,0,32,1326.0,10.0,10.0,1228.84
1,0,0,33,1041.0,1.0,1.0,1228.84
2,0,0,35,247.0,4.0,4.0,1228.84
3,0,0,43,221.0,5.0,5.0,1228.84
4,0,0,51,128.5,1.0,1.0,1228.84
5,0,0,61,195.0,1.0,1.0,1228.84
6,0,0,75,76.0,1.0,1.0,1228.84
7,0,0,88,76.0,2.0,2.0,1228.84
8,0,0,95,193.0,9.0,9.0,1228.84
9,0,0,96,70.0,1.0,1.0,1228.84


In [95]:
df3['shop_id_sum_item_cnt_day'] = df2.groupby(['date_block_num','shop_id'])['item_cnt_day'].transform('sum')

In [96]:
df3.sample(20)

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day
1409588,27,50,12974,249.0,3.0,3.000000,1777.77,1011.0
508587,8,52,19266,198.0,2.0,2.000000,1975.84,2799.0
1606173,33,57,8607,449.0,7.0,2.333333,1439.02,1444.0
366751,6,22,2241,399.0,2.0,1.000000,1301.12,8105.0
742086,13,4,14654,199.0,3.0,3.000000,470.81,960.0
119122,1,54,10689,2397.0,2.0,2.000000,1423.11,2665.0
234129,3,54,13037,499.0,1.0,1.000000,939.35,1370.0
1092972,20,43,4271,1699.0,1.0,1.000000,1406.08,1086.0
1262967,24,17,5509,299.0,1.0,1.000000,3000.63,2345.0
733402,12,54,14690,199.0,1.0,1.000000,2848.78,5611.0


In [97]:
df3['shop_id_mean_item_cnt_day'] = df2.groupby(['date_block_num','shop_id'])['item_cnt_day'].transform('mean')
df3['shop_id_mean_item_cnt_day'] = round(df3['shop_id_mean_item_cnt_day'],2)
df3.head()

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day,shop_id_mean_item_cnt_day
0,0,0,32,1326.0,10.0,10.0,1228.84,5578.0,2.31
1,0,0,33,1041.0,1.0,1.0,1228.84,5578.0,2.31
2,0,0,35,247.0,4.0,4.0,1228.84,5578.0,2.31
3,0,0,43,221.0,5.0,5.0,1228.84,5578.0,2.31
4,0,0,51,128.5,1.0,1.0,1228.84,5578.0,2.31


In [103]:
df3['item_category_id_avg_item_price'] = df2.groupby(['date_block_num','shop_id','item_category_id'])['total_sales'].transform('mean')
df3['item_category_id_avg_item_price'] = round(df3['item_category_id_avg_item_price'],2)
df3.sample(20)

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day,shop_id_mean_item_cnt_day,item_category_id_avg_item_price
906187,16,31,19420,349.000000,1.0,1.000000,1772.95,1236.0,1.78,10946.00
389987,6,44,10687,1000.000000,1.0,1.000000,1584.55,2676.0,1.82,458.92
1515725,31,5,18129,329.000000,1.0,1.000000,2104.95,726.0,1.70,9771.05
512921,8,56,1610,499.000000,1.0,1.000000,1845.94,2811.0,2.13,334.75
820633,14,43,12208,1499.000000,1.0,1.000000,1959.01,9749.0,2.89,465.78
883880,16,5,1527,999.000000,7.0,2.333333,2722.27,2130.0,1.99,10596.50
1057204,19,52,4904,1596.000000,2.0,2.000000,1680.52,1249.0,2.07,1725.11
1115669,21,18,18476,398.000000,2.0,2.000000,1819.32,1559.0,1.83,1359.18
615886,10,56,8436,249.000000,1.0,1.000000,6300.72,2926.0,3.58,274.85
214441,3,31,8883,1299.000000,1.0,1.000000,897.70,1850.0,1.49,2029.87


In [104]:
df3['item_category_id_sum_item_cnt_day'] = df2.groupby(['date_block_num','shop_id','item_category_id'])['item_cnt_day'].transform('sum')
df3['item_category_id_avg_item_cnt_day'] = df2.groupby(['date_block_num','shop_id','item_category_id'])['item_cnt_day'].transform('mean')
df3['item_category_id_sum_item_cnt_day'] = round(df3['item_category_id_sum_item_cnt_day'],2)
df3['item_category_id_avg_item_cnt_day'] = round(df3['item_category_id_avg_item_cnt_day'],2)
df3.sample(20)

,date_block_num,shop_id,item_id,item_id_avg_item_price,item_id_sum_item_cnt_day,item_id_avg_item_cnt_day,shop_id_avg_item_price,shop_id_sum_item_cnt_day,shop_id_mean_item_cnt_day,item_category_id_avg_item_price,item_category_id_sum_item_cnt_day,item_category_id_avg_item_cnt_day
464848,7,58,4006,299.0,1.0,1.000000,1425.61,7539.0,2.60,476.83,900.0,1.76
970177,17,56,10576,249.0,1.0,1.000000,1999.59,7846.0,2.61,529.31,603.0,1.44
1601751,33,48,8547,2398.0,2.0,2.000000,1608.32,1327.0,1.68,1725.73,85.0,1.39
1127022,21,30,19439,7347.5,2.0,2.000000,1910.59,1458.0,1.77,1555.63,70.0,1.21
547539,9,35,3705,1499.0,1.0,1.000000,1828.35,7837.0,2.57,1524.93,73.0,1.66
199427,3,21,11251,298.0,2.0,2.000000,1147.46,3781.0,1.96,1513.91,567.0,3.66
437705,7,31,1441,1000.0,4.0,1.333333,880.52,891.0,1.43,712.60,75.0,1.44
322199,5,31,4602,598.0,47.0,3.916667,3122.79,1720.0,2.85,2086.51,124.0,3.02
289825,4,56,13782,149.0,3.0,3.000000,1488.32,9429.0,2.68,1311.09,44.0,1.91
310062,5,21,21984,299.0,2.0,2.000000,858.50,2147.0,1.56,1279.27,343.0,2.60


In [105]:
df_final =df3